1.1 Import Data and Required Packages

In [51]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [52]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

Import CSV data as pandas dataframe

In [53]:
df = pd.read_csv("stud.csv")

In [54]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [55]:
df.shape

(1000, 8)

Preparing X and y variables

In [56]:
X = df.drop(columns = ['math_score'], axis = 1)
y = df['math_score']

In [57]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

In [58]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [59]:
ohe = OneHotEncoder()
scale = StandardScaler()

In [60]:
preprocessor = ColumnTransformer(
    [
    ("OneHotEncoder", ohe, cat_features),
    ("StandardScaler", scale, num_features),
    ]
)

In [61]:
X = preprocessor.fit_transform(X)

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

Create evaluation metrics

In [64]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)

    return mae, rmse, r2_square

In [65]:
models ={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "Catboost": CatBoostRegressor(verbose = False),
    "AdaBoost": AdaBoostRegressor(),
}
model_list = []
r2_list = []

In [66]:
#train model

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

#make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

#evaluate train and test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for training set")
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('-------------------------------')

    print("Model performance for test set")
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)

#    print('='*35)
#    print('\n')

Linear Regression
Model performance for training set
- Mean Absolute Error: 4.2715
- Root Mean Squared Error: 5.3293
- R2 Score: 0.8740
-------------------------------
Model performance for test set
- Mean Absolute Error: 4.2222
- Root Mean Squared Error: 5.4252
- R2 Score: 0.8790
Lasso
Model performance for training set
- Mean Absolute Error: 5.2063
- Root Mean Squared Error: 6.5938
- R2 Score: 0.8071
-------------------------------
Model performance for test set
- Mean Absolute Error: 5.1579
- Root Mean Squared Error: 6.5197
- R2 Score: 0.8253
Ridge
Model performance for training set
- Mean Absolute Error: 4.2650
- Root Mean Squared Error: 5.3233
- R2 Score: 0.8743
-------------------------------
Model performance for test set
- Mean Absolute Error: 4.2111
- Root Mean Squared Error: 5.3904
- R2 Score: 0.8806
K-Neighbors
Model performance for training set
- Mean Absolute Error: 4.5168
- Root Mean Squared Error: 5.7079
- R2 Score: 0.8555
-------------------------------
Model performanc

Results

In [67]:
pd.DataFrame(list(zip(model_list, r2_list)), columns =['Model Name', 'R2_Score']).sort_values(by = ['R2_Score'], ascending = False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.879046
7,Catboost,0.851632
8,AdaBoost,0.851623
5,Random Forest,0.850577
1,Lasso,0.825320
6,XGBRegressor,0.821589
3,K-Neighbors,0.783813
4,DecisionTree,0.739047


Linear Regression